In [1]:
# Import base librares
import pandas as pd
import numpy as np
import os
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 


In [57]:
# Download data files
data_folder = Path('../readonly/final_project_data')

transactions    = pd.read_csv(data_folder/'sales_train.csv.gz')
items           = pd.read_csv(data_folder/'items.csv')
item_categories = pd.read_csv(data_folder/'item_categories.csv')
shops           = pd.read_csv(data_folder/'shops.csv')

test = pd.read_csv(data_folder/'test.csv.gz', index_col='ID')

In [3]:
# Base data observation
print('Transactions shape:', transactions.shape)
transactions.head()

Transactions shape: (2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
print('items shape:', items.shape)
items.head()

items shape: (22170, 3)


,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [5]:
print('item_categories shape:', item_categories.shape)
item_categories.head()

item_categories shape: (84, 2)


,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [6]:
print('shops shape:', shops.shape)
shops.head()

shops shape: (60, 2)


,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [58]:
print(test.info())
print('Test date shape:', shops.shape)
test.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214200 entries, 0 to 214199
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   shop_id  214200 non-null  int64
 1   item_id  214200 non-null  int64
dtypes: int64(2)
memory usage: 4.9 MB
None
Test date shape: (60, 2)


,shop_id,item_id
ID,,
0,5,5037
1,5,5320
2,5,5233
3,5,5232
4,5,5268


In [8]:
# Merge into one table
transactions.date = pd.to_datetime(transactions.date, yearfirst=True)
data = transactions.merge(items).merge(item_categories).merge(shops)
# Calculate income from transaction
# data['income'] = data['item_price'] * data['item_cnt_day']
# Number of month since year 2013
data['mdate'] = ((data['date'].dt.year-2013)*12)+data['date'].dt.month
# Add year, month and day rows
# data['year'] = data['date'].dt.year
# data['month'] = data['date'].dt.month
# data['day'] = data['date'].dt.day
data

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name,mdate
0,2013-02-01,0,59,22154,999.0,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",2
1,2013-04-26,3,59,944,150.0,1.0,2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",4
2,2013-06-26,5,59,944,199.5,1.0,2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",6
3,2013-07-20,6,59,944,199.5,1.0,2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",7
4,2013-09-14,8,59,944,299.0,2.0,2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",9
...,...,...,...,...,...,...,...,...,...,...,...
2935844,2015-10-22,33,55,13093,250.0,1.0,Карта оплаты Windows: 250 рублей [Цифровая вер...,36,Карты оплаты - Windows (Цифра),Цифровой склад 1С-Онлайн,34
2935845,2015-09-21,32,55,13091,1000.0,1.0,Карта оплаты Windows: 1000 рублей [Цифровая ве...,36,Карты оплаты - Windows (Цифра),Цифровой склад 1С-Онлайн,33
2935846,2015-09-16,32,55,13094,2500.0,1.0,Карта оплаты Windows: 2500 рублей [Цифровая ве...,36,Карты оплаты - Windows (Цифра),Цифровой склад 1С-Онлайн,33
2935847,2015-09-22,32,55,13094,2500.0,2.0,Карта оплаты Windows: 2500 рублей [Цифровая ве...,36,Карты оплаты - Windows (Цифра),Цифровой склад 1С-Онлайн,33


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935849 entries, 0 to 2935848
Data columns (total 11 columns):
 #   Column              Dtype         
---  ------              -----         
 0   date                datetime64[ns]
 1   date_block_num      int64         
 2   shop_id             int64         
 3   item_id             int64         
 4   item_price          float64       
 5   item_cnt_day        float64       
 6   item_name           object        
 7   item_category_id    int64         
 8   item_category_name  object        
 9   shop_name           object        
 10  mdate               int64         
dtypes: datetime64[ns](1), float64(2), int64(5), object(3)
memory usage: 268.8+ MB


%%time
# Calc normalized monthly sum of sales in each shop/item
d_sum = data.groupby(['shop_id', 'item_id', 'mdate'])['item_cnt_day'].sum()
d_sum_norm = d_sum/d_sum.max()*20
d_sum_norm


# Make the fiture - previous_value_benchmark as 
d_sum_idx = pd.MultiIndex.from_product([np.arange(data['shop_id'].max()+1)
                                        , np.arange(data['item_id'].max()+1)
                                        , np.arange(data['mdate'].max()+1)])
d_sum_indexed = pd.DataFrame(index=d_sum_idx, data=d_sum_norm)
d_sum_indexed = d_sum_indexed.shift(periods=-1)
d_sum_indexed.fillna(value=0, inplace=True)
d_sum_indexed.reset_index(inplace=True)
d_sum_indexed.columns=['shop_id', 'item_id', 'mdate', 'previous_value_benchmark']
d_sum_indexed

In [15]:
# data.merge(how='left', right=d_sum_indexed, on=['shop_id', 'item_id', 'mdate'])

In [64]:
%%time
# Calc normalized monthly sum of sales in each shop/item
d_sum = data[data['mdate']==34].groupby(['shop_id', 'item_id'])['item_cnt_day'].sum()
d_sum_norm = d_sum/d_sum.max()*20
d_sum_norm

CPU times: user 33.6 ms, sys: 5.34 ms, total: 39 ms
Wall time: 83.1 ms


shop_id  item_id
2        31         0.008905
         67         0.008905
         486        0.017809
         787        0.008905
         792        0.008905
                      ...   
59       22088      0.017809
         22091      0.008905
         22100      0.008905
         22102      0.008905
         22118      0.008905
Name: item_cnt_day, Length: 34101, dtype: float64

In [65]:
# Make the fiture - previous_value_benchmark as 
d_sum_idx = pd.MultiIndex.from_product([np.arange(data['shop_id'].max()+1)
                                        , np.arange(data['item_id'].max()+1)])
d_sum_indexed = pd.DataFrame(index=d_sum_idx, data=d_sum_norm)
# d_sum_indexed = d_sum_indexed.shift(periods=-1)
d_sum_indexed.fillna(value=0, inplace=True)
d_sum_indexed.reset_index(inplace=True)
d_sum_indexed.columns=['shop_id', 'item_id', 'item_cnt_month']
d_sum_indexed

,shop_id,item_id,item_cnt_month
0,0,0,0.0
1,0,1,0.0
2,0,2,0.0
3,0,3,0.0
4,0,4,0.0
...,...,...,...
1330195,59,22165,0.0
1330196,59,22166,0.0
1330197,59,22167,0.0
1330198,59,22168,0.0


In [66]:
submission = test.merge(d_sum_indexed, how='left', sort=False)
submission.index.name='ID'
submission.drop(columns=['shop_id', 'item_id']).to_csv('submission.csv', index=True)
# submission.to_csv('submission.csv', index=True)
!ls -l submission.csv
submission

-rw-r--r-- 1 jovyan users 2728058 Aug  8 14:41 submission.csv


,shop_id,item_id,item_cnt_month
ID,,,
0,5,5037,0.000000
1,5,5320,0.000000
2,5,5233,0.000000
3,5,5232,0.000000
4,5,5268,0.000000
...,...,...,...
214195,45,18454,0.017809
214196,45,16188,0.000000
214197,45,15757,0.000000


In [67]:
!kaggle competitions submit -c competitive-data-science-final-project -f submission.csv -m "Test kaggle submit"

401 - Unauthorized
